In [34]:
import json
import os
import numpy as np
import pandas as pd
import time
import io
import distutils.dir_util
from collections import Counter
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import os
from google.colab import drive
from torchsummary import summary
drive.mount('/content/Mydrive')

Drive already mounted at /content/Mydrive; to attempt to forcibly remount, call drive.mount("/content/Mydrive", force_remount=True).


In [3]:
print(torch.__version__)

1.6.0+cu101


In [4]:
cd ./Mydrive/My Drive/kakaomelon 

/content/Mydrive/My Drive/kakaomelon


In [5]:
!ls

arena_data	   genre_most_popular.py  results.json	  train.json
arena_util.py	   most_popular.py	  song_meta.json  val.json
evaluate.py	   NanumGothic.ttf	  split_data.py
genre_gn_all.json  __pycache__		  test.json


In [6]:
song_meta = pd.read_json("./song_meta.json")
song_meta.head(5)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [7]:
train = pd.read_json("./arena_data/orig/train.json")

In [8]:
test = pd.read_json("./arena_data/questions/val.json")

In [9]:
print(train.shape,test.shape)

(92056, 6) (23015, 6)


#preprocessing 

In [10]:
train['istrain'] = 1 
test['istrain'] = 0 

In [11]:
n_train = len(train) 
n_test = len(test) 
print(n_train,n_test)

92056 23015


Train + Test 

In [12]:
plylst = pd.concat([train, test], ignore_index=True)
plylst.shape

(115071, 7)

nid & id mapping 

In [13]:
plylst["nid"] = range(n_train + n_test)
plylst_nid_id = dict(zip(plylst["nid"],plylst["id"]))

Songs cnt 

In [14]:
#song
plylst_song = plylst['songs']
song_counter = Counter([sg for sgs in plylst_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}
song_dict=dict(filter(lambda x : x[1]>200,song_dict.items()))

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
  song_id_sid[t] = i
  song_sid_id[i] = t

n_songs = len(song_dict)
print("Number of songs:",n_songs)

Number of songs: 3087


Tag cnt 

In [15]:
#tag
from collections import Counter
plylst_tag = plylst['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])

tag_dict = {x: tag_counter[x] for x in tag_counter}
#print(tag_counter)
tag_dict=dict(filter(lambda x:x[1]>5,tag_dict.items()))

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
  tag_id_tid[t] = i
  tag_tid_id[i] = t

n_tags = len(tag_dict)
print("Number of tags:",n_tags)

Number of tags: 3759


Song,Tag 속하는지 여부에 따라 mapping 

In [16]:
plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

합쳐주고 다시 split 

In [17]:
plylst_use = plylst[['istrain','nid','songs_id','tags_id']]
plylst_use.loc[:,'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')


plylst_train = plylst_use.iloc[:n_train,:]
plylst_test = plylst_use.iloc[n_train:,:]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [20]:
test = plylst_test

Train 

In [18]:
plylst_train.head()

,istrain,songs_id,tags_id,num_songs,num_tags
nid,,,,,
0,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3]",41,4
1,1,[41],[4],1,1
2,1,[],[5],0,1
3,1,"[42, 43]","[6, 7]",2,2
4,1,"[44, 45, 46, 47, 48]","[0, 1, 8]",5,3


#Dstack songs 

In [22]:
plylst_train['id']=range(n_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
plylst_train_map=plylst_train[['id','songs_id']]
plylst_train_map_unnest = np.dstack(
    (
        np.repeat(plylst_train_map.id.values, list(map(len, plylst_train_map.songs_id))), 
        np.concatenate(plylst_train_map.songs_id.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_train_song = pd.DataFrame(data = plylst_train_map_unnest[0], columns = plylst_train_map.columns)

In [29]:
plylst_train_song.shape

(1091942, 2)

In [30]:
plylst_train_song.head(5)

,id,songs_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,0.0,4.0


#Dstack tags 

In [31]:
plylst_train_map=plylst_train[['id','tags_id']]
plylst_train_map_unnest = np.dstack(
    (
        np.repeat(plylst_train_map.id.values, list(map(len, plylst_train_map.tags_id))), 
        np.concatenate(plylst_train_map.tags_id.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_train_tag = pd.DataFrame(data = plylst_train_map_unnest[0], columns = plylst_train_map.columns)

In [32]:
plylst_train_tag.shape

(348105, 2)

In [33]:
plylst_train_tag.head(5)

,id,tags_id
0,0.0,0.0
1,0.0,1.0
2,0.0,2.0
3,0.0,3.0
4,1.0,4.0


#Neural CF 

In [42]:
class NeuralCF(nn.Module):
  def __init__(self,num_users,num_songs,embedding_size,**kwargs):
    super(NeuralCF,self).__init__(**kwargs)
    self.user_embedding = nn.Embedding(num_users,embedding_size)
    self.song_embedding = nn.Embedding(num_songs,embedding_size)
    self.fc1=nn.Linear(embedding_size,256)
    self.fc2=nn.Linear(256,512)
    self.fc3=nn.Linear(512,1024)
  def forward(self,x,y):
    x=self.user_embedding(x)
    y=self.song_embedding(y)
    x=torch.cat([x,y], dim=-1)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.relu(self.fc3(x))
    return F.sigmoid(x) 

model=NeuralCF(n_train,n_songs,64)

model

NeuralCF(
  (user_embedding): Embedding(92056, 64)
  (song_embedding): Embedding(3087, 64)
  (fc1): Linear(in_features=64, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
)